# initialization
#### 3 dictionary for unygram of each poet
#### 3 dictionary for bygram of each poet

In [121]:
hafez={}
ferdowsi={}
molavi={}
hafez2={}
ferdowsi2={}
molavi2={}

# unygram

In [122]:
with open("train_set/hafez_train.txt", encoding="utf8") as reader:
    for line in reader:
        line=line.replace('،',' ')
        line=line.split()
        for word in line:
            if word in hafez:
                hafez[word]=hafez.get(word)+1
            else:
                hafez[word]=1
                

In [123]:
with open("train_set/ferdowsi_train.txt", encoding="utf8") as reader:
    for line in reader:
        line=line.replace('،',' ')
        line=line.split()
        for word in line:
            if word in ferdowsi:
                ferdowsi[word]=ferdowsi.get(word)+1
            else:
                ferdowsi[word]=1

In [124]:
with open("train_set/molavi_train.txt", encoding="utf8") as reader:
    for line in reader:
        line=line.replace('،',' ')
        line=line.split()
        for word in line:
            if word in molavi:
                molavi[word]=molavi.get(word)+1
            else:
                molavi[word]=1

# bygram

In [125]:

with open("train_set/ferdowsi_train.txt", encoding="utf8") as reader:
    for line in reader:
        line=line.replace('،',' ')
        line=line.split()
        for i in range(len(line)):
            if i == len(line)-1:
                twoWords=line[i]+' <>'
            else:
                twoWords=line[i]+' '+line[i+1]
            if twoWords in hafez2:
                ferdowsi2[twoWords]=hafez2.get(twoWords)+1
            else:
                ferdowsi2[twoWords]=1

In [126]:
with open("train_set/hafez_train.txt", encoding="utf8") as reader:
    for line in reader:
        line=line.replace('،',' ')
        line=line.split()
        for i in range(len(line)):
            if i == len(line)-1:
                twoWords=line[i]+' <>'
            else:
                twoWords=line[i]+' '+line[i+1]
            if twoWords in hafez2:
                hafez2[twoWords]=hafez2.get(twoWords)+1
            else:
                hafez2[twoWords]=1
    

In [127]:
with open("train_set/molavi_train.txt", encoding="utf8") as reader:
    for line in reader:
        line=line.replace('،',' ')
        line=line.split()
        for i in range(len(line)):
            if i == len(line)-1:
                twoWords=line[i]+' <>'
            else:
                twoWords=line[i]+' '+line[i+1]
            if twoWords in hafez2:
                molavi2[twoWords]=hafez2.get(twoWords)+1
            else:
                molavi2[twoWords]=1

# cleans the data
### omit words with les frequency than two times

In [128]:
new_dict = {}
for key, value in hafez.items():
    if value > 2:
        new_dict[key] = value
hafez = new_dict

In [129]:
new_dict = {}
for key, value in ferdowsi.items():
    if value > 2:
        new_dict[key] = value
ferdowsi = new_dict

In [130]:
new_dict = {}
for key, value in molavi.items():
    if value > 2:
        new_dict[key] = value
molavi = new_dict

# Read the Test

In [176]:
poets=[]
poems=[]

In [177]:
with open("test_set/test_case.txt", encoding="utf8") as reader:
    for line in reader:
        s=line.split('\t')
        poets.append(s[0])
        poems.append(s[1])


#  backoff calculate probability of a wordin entern unygram and bygram

In [178]:
def backoff(w1,w2,unygram,bygram,lambda1,lambda2,lambda3,beta):
    resault=0
    ww=w1+' '+w2
    
    if ww in bygram and w1 in unygram:
        
        resault = lambda1*bygram.get(ww)/unygram.get(w1)
    if w1 in unygram:
        resault+=lambda2*unygram.get(w1)/len(unygram.keys())
    resault+=lambda3*beta
    return resault

### Probability of a poem to be for a poet

In [179]:
def howMuchIn(poem,unigram,bygram,lambda1,lambda2,lambda3,beta):
    poem=poem.split()
    prob=0

    for i in range(len(poem)):
        if i==len(poem)-1:
            prob+=backoff(poem[i],'<>',unigram,bygram,lambda1,lambda2,lambda3,beta)
        else:
            prob+=backoff(poem[i],poem[i+1],unigram,bygram,lambda1,lambda2,lambda3,beta)
    
    return prob

#  poem is for which poet 

In [180]:
def whichOne(poem,lambda1,lambda2,lambda3,beta):
    probF = howMuchIn(poem,ferdowsi,ferdowsi2,lambda1[0],lambda2[0],lambda3[0],beta[0])
    probH = howMuchIn(poem,hafez,hafez2,lambda1[1],lambda2[1],lambda3[1],beta[1])
    probM = howMuchIn(poem,molavi,molavi2,lambda1[2],lambda2[2],lambda3[2],beta[2])
    m=max(probF,probH,probM)
    #print(probF,probH,probM)
    if probF==m:
        return '1'
    if probH==m:
        return '2'
    return '3'
            

In [189]:
len(poets)

138

# dataset is not balance

In [136]:
print(len(ferdowsi2.keys()),len(hafez2.keys()),len(molavi2.keys()))
            
print(len(ferdowsi.keys()),len(hafez.keys()),len(molavi.keys()))

28006 37632 34916
2153 2623 2374


# lambda trainer

In [181]:
def maxLambda(lambda1,lambda2,lambda3,beta):
    val=[[0,0,0],[0,0,0],[0,0,0]]
    for i in range(len(poems)):
        temp = whichOne(poems[i],lambda1,lambda2,lambda3,beta)
        
        if poets[i] == temp:
            val[1][int(temp)-1]+=1
        else:
            val[2][int(temp)-1]+=1
        val[0][int(poets[i])-1]+=1

    return val

In [190]:
def p(a):
    return 100*sum(a)/138

#  compare

In [191]:
_,t,f = maxLambda([0.99,0.99,0.99],[0.005,0.005,0.003],[0.01,0.0051,0.007],[0.1,0.0001,0.001])
print(_,t,f)
print(p(t))

[50, 35, 53] [17, 24, 27] [10, 32, 28]
49.27536231884058


In [192]:
_,t,f = maxLambda([0.15,0.15,0.15],[0.75,0.75,0.75],[0.01,0.0051,0.007],[0.1,0.0001,0.001])
print(_,t,f)
print(p(t))

[50, 35, 53] [18, 16, 36] [17, 30, 21]
50.72463768115942


In [193]:
_,t,f = maxLambda([0.75,0.75,0.75],[0.25,0.15,0.15],[0.01,0.0001,0.007],[0.1,0.0001,0.001])
print(_,t,f)
print(p(t))

[50, 35, 53] [28, 22, 24] [23, 24, 17]
53.6231884057971


In [194]:
_,t,f = maxLambda([0.59,0.6,0.59],[0.39,0.4,0.39],[0.003,0.001,0.01],[0.02,0.001,0.001])
print(_,t,f)
print(p(t))

[50, 35, 53] [16, 25, 36] [8, 34, 19]
55.79710144927536


In [195]:
_,t,f = maxLambda([0.005,0.005,0.003],[0.99,0.99,0.99],[0.01,0.0051,0.007],[0.1,0.0001,0.001])
print(_,t,f)
print(p(t))

[50, 35, 53] [25, 15, 37] [19, 27, 15]
55.79710144927536


In [196]:
_,t,f = maxLambda([0.39,0.4,0.39],[0.59,0.6,0.59],[0.003,0.001,0.01],[0.02,0.001,0.001])
print(_,t,f)
print(p(t))

[50, 35, 53] [18, 23, 38] [8, 32, 19]
57.2463768115942


In [197]:
_,t,f = maxLambda([0.49,0.4,0.39],[0.59,0.6,0.59],[0.003,0.001,0.01],[0.02,0.001,0.001])
print(_,t,f)
print(p(t))

[50, 35, 53] [19, 23, 36] [10, 31, 19]
56.52173913043478


In [198]:
_,t,f = maxLambda([0.50,0.65,0.50],[0.49,0.349,0.49],[0.01,0.001,0.01],[0.02,0.001,0.001])
print(_,t,f)
print(p(t))

[50, 35, 53] [23, 20, 37] [10, 24, 24]
57.971014492753625


In [199]:
_,t,f = maxLambda([0.9,0.9,0.9],[0.05,0.05,0.05],[0.05,0.05,0.05],[0.001,0.00001,0.001])
print(_,t,f)
print(p(t))

[50, 35, 53] [15, 25, 31] [4, 34, 29]
51.44927536231884
